In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
a = pd.read_csv('data/Amazon.csv',  encoding='unicode_escape')
a.columns = [c.upper() for c in a.columns]
amazon = session.create_dataframe(a)
amazon.write.mode("overwrite").save_as_table("amazon_items")

g = pd.read_csv('data/GoogleProducts.csv',  encoding='unicode_escape')
g.columns = [c.upper() for c in g.columns]
google = session.create_dataframe(g)
google.write.mode("overwrite").save_as_table("google_items")

In [ ]:
select * from amazon_items
limit 5;

First create a lookup table representing the largest table.  We want 2 columns, an ID column and a column with all the information related to the ID/Product

In [ ]:
UPDATE amazon_items
SET 
    title = COALESCE(title, 'NA'),
    DESCRIPTION = COALESCE(DESCRIPTION, 'NA'),
    manufacturer = COALESCE(manufacturer, 'NA'),
    Price = COALESCE(Price, 'NA')
WHERE 
    title IS NULL OR DESCRIPTION IS NULL OR manufacturer IS NULL;

create or replace table amazon_lookup as
select ID, 'Name: '||title||', Description: '||DESCRIPTION ||', Manufacturer: '||manufacturer||', Price: '||Price as ITEM
from amazon_items;

In [ ]:
select * from amazon_lookup;

In [ ]:
current_wh = session.get_current_warehouse()
current_db = session.get_current_database()
current_schema = session.get_current_schema()

In [ ]:
session.sql(f'''CREATE OR REPLACE CORTEX SEARCH SERVICE PRODUCT_LOOKUP
  ON ITEM
  ATTRIBUTES ID
  WAREHOUSE = {current_wh}
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  AS (
    SELECT
        ID,
        ITEM
    FROM amazon_lookup)''')

In [ ]:
UPDATE google_items
SET 
    name = COALESCE(name, 'NA'),
    DESCRIPTION = COALESCE(DESCRIPTION, 'NA'),
    manufacturer = COALESCE(manufacturer, 'NA'),
    Price = COALESCE(Price, 'NA')
WHERE 
    name IS NULL OR DESCRIPTION IS NULL OR manufacturer IS NULL;
    
create or replace table google_lookup as
select ID, 'Name: '||name||', Description: '||DESCRIPTION ||', Manufacturer: '||manufacturer||', Price: '||Price as ITEM
from google_items;

In [ ]:
select * from google_lookup
limit 10;

In [ ]:
create or replace table batch_search_results as
SELECT
  q.ID as google_id,
  q.ITEM as google_desc,
  s.ID as amazon_id,
  s.ITEM as amazon_desc,
  parse_json(METADATA$RESULT_DETAIL) as scores,
  scores:scores:cosine_similarity::float as cos_sim
FROM google_lookup AS q,
LATERAL CORTEX_SEARCH_BATCH(
  service_name => 'PRODUCT_LOOKUP',
  query => q.ITEM,
  limit        => 2
) AS s;

In [ ]:
select *
from batch_search_results
where COS_SIM > .9
limit 5;

In [ ]:
create or replace table matches_w_claude as
SELECT amazon_id, 
google_id, 
amazon_desc, 
google_desc, 
cos_sim,
SNOWFLAKE.CORTEX.AI_COMPLETE(
    'claude-4-sonnet',
        CONCAT('You are responsible for identifying if two products are the EXACT same product but sold on two different websites.
Given the descriptions of the two products from each website, return a 1 if they are likely the same product, 0 if they are not.
Item 1 Description: ', google_desc,
' Item 2 Description: ', amazon_desc,
'Respond only with a JSON object in the following format: {
  "Match": 1,
  "Reasoning": "Concise explanation here"
}')
) as match from batch_search_results
where cos_sim > .50
;

In [ ]:
create or replace table snowflake_matches_claude
as
select
amazon_id, 
google_id, 
amazon_desc,
google_desc,
try_parse_json(match):Match::int as match,
try_parse_json(match):Reasoning::varchar as reasoning
from matches_w_claude
where try_parse_json(match):Match::int = 1;

In [ ]:
select * from snowflake_matches_claude
limit 10;

In [ ]:
m = pd.read_csv('data/Amzon_GoogleProducts_perfectMapping.csv',  encoding='unicode_escape')
m.columns = [c.upper() for c in m.columns]
mapping = session.create_dataframe(m)
mapping.write.mode("overwrite").save_as_table("amazon_google_mapping")

In [ ]:
create or replace table batch_search_results_cutoff
as
select * from
batch_search_results
where COS_SIM > .6;

In [ ]:
WITH
true_positives AS (
    SELECT COUNT(*) AS tp
    FROM batch_search_results_cutoff sm
    INNER JOIN amazon_google_mapping agm
        ON sm.amazon_id = agm.idamazon AND sm.google_id = agm.idgooglebase
),
false_positives AS (
    SELECT COUNT(*) AS fp
    FROM batch_search_results_cutoff sm
    LEFT JOIN amazon_google_mapping agm
        ON sm.amazon_id = agm.idamazon AND sm.google_id = agm.idgooglebase
    WHERE agm.idamazon IS NULL
),
false_negatives AS (
    SELECT COUNT(*) AS fn
    FROM amazon_google_mapping agm
    LEFT JOIN batch_search_results_cutoff sm
        ON sm.amazon_id = agm.idamazon AND sm.google_id = agm.idgooglebase
    WHERE sm.amazon_id IS NULL
),
metrics AS (
    SELECT
        tp,
        fp,
        fn,
        CASE WHEN (tp + fp) = 0 THEN 0 ELSE tp::FLOAT / (tp + fp) END AS precision,
        CASE WHEN (tp + fn) = 0 THEN 0 ELSE tp::FLOAT / (tp + fn) END AS recall
    FROM true_positives, false_positives, false_negatives
)
SELECT
    tp,
    fp,
    fn,
    precision,
    recall,
    CASE
        WHEN (precision + recall) = 0 THEN 0
        ELSE 2 * (precision * recall) / (precision + recall)
    END AS f1_score
FROM metrics;

In [ ]:
WITH
true_positives AS (
    SELECT COUNT(*) AS tp
    FROM snowflake_matches_claude sm
    INNER JOIN amazon_google_mapping agm
        ON sm.amazon_id = agm.idamazon AND sm.google_id = agm.idgooglebase
),
false_positives AS (
    SELECT COUNT(*) AS fp
    FROM snowflake_matches_claude sm
    LEFT JOIN amazon_google_mapping agm
        ON sm.amazon_id = agm.idamazon AND sm.google_id = agm.idgooglebase
    WHERE agm.idamazon IS NULL
),
false_negatives AS (
    SELECT COUNT(*) AS fn
    FROM amazon_google_mapping agm
    LEFT JOIN snowflake_matches_claude sm
        ON sm.amazon_id = agm.idamazon AND sm.google_id = agm.idgooglebase
    WHERE sm.amazon_id IS NULL
),
metrics AS (
    SELECT
        tp,
        fp,
        fn,
        CASE WHEN (tp + fp) = 0 THEN 0 ELSE tp::FLOAT / (tp + fp) END AS precision,
        CASE WHEN (tp + fn) = 0 THEN 0 ELSE tp::FLOAT / (tp + fn) END AS recall
    FROM true_positives, false_positives, false_negatives
)
SELECT
    tp,
    fp,
    fn,
    precision,
    recall,
    CASE
        WHEN (precision + recall) = 0 THEN 0
        ELSE 2 * (precision * recall) / (precision + recall)
    END AS f1_score
FROM metrics;